# LightningDataModule
datamodule是一个能够分享、重用的类，封装了处理数据所需的所有步骤。

datamodule封装了pytorch中数据处理的五个步骤：
1. 下载/标记/处理
2. 清理并且（或者）存储到硬盘
3. 载入已有Dataset
4. 应用转换（旋转，标记等）
5. 包装到DataLoader中

这些类可以被分享和使用。

In [1]:
import pytorch_lightning as pl
a = 1

In [ ]:
from pl_bolts.datamodules import CIFAR10DataModule, ImagenetDataModule

model = LitClassifier()
trainer = Trainer()

imagenet = ImagenetDataModule()
trainer.fit(model, datamodule=imagenet)

cifar10 = CIFAR10DataModule()
trainer.fit(model, datamodule=cifar10)

一个DataModule是一个集合，包括train_dataloader(s),val_dataloader(s),test_dataloader(s)和predict_dataloader(s),以及相应的转换和数据下载处理步骤。

以下是常规的pytorch数据处理代码

In [ ]:
# regular PyTorch
test_data = MNIST(my_path, train=False, download=True)
predict_data = MNIST(my_path, train=False, download=True)
train_data = MNIST(my_path, train=True, download=True)
train_data, val_data = random_split(train_data, [55000, 5000])

train_loader = DataLoader(train_data, batch_size=32)
val_loader = DataLoader(val_data, batch_size=32)
test_loader = DataLoader(test_data, batch_size=32)
predict_loader = DataLoader(predict_data, batch_size=32)

等价的DataModule只是将这些代码重新组织，但是能够让它们在不同的项目中使用。

In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = "path/to/dir", batch_size: int = 32):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

    def setup(self, stage: str):
        self.mnist_test = MNIST(self.data_dir, train=False)
        self.mnist_predict = MNIST(self.data_dir, train=False)
        mnist_full = MNIST(self.data_dir, train=True)
        self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.mnist_predict, batch_size=self.batch_size)

    def teardown(self, stage: str):
        # Used to clean-up when the run is finished
        ...

DataModule需要定义以下的方法：
- prepare_data(下载，标记数据等)
- setup（分割，定义数据集等）
- train_dataloader
- val_dataloader
- test_dataloader
- predict_dataloader

### prepare_data
`setup()`会在`prepare_data()`之后调用，确保数据准备完毕。不建议在`prepare_data()`中分配状态（如`self.x=y`），其仅负责存储数据。

In [ ]:
class MNISTDataModule(pl.LightningDataModule):
    def prepare_data(self):
        # download
        MNIST(os.getcwd(), train=True, download=True, transform=transforms.ToTensor())
        MNIST(os.getcwd(), train=False, download=True, transform=transforms.ToTensor())


### setup
有时我们想要在每个GPU上对数据做一些操作，`setup()`用来做以下操作：
- 创建字典
- 分割数据集
- 创建数据集
- 应用数据转换
- 等等

In [ ]:
import pytorch_lightning as pl


class MNISTDataModule(pl.LightningDataModule):
    def setup(self, stage: str):
        # Assign Train/val split(s) for use in Dataloaders
        if stage == "fit":
            mnist_full = MNIST(self.data_dir, train=True, download=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])

        # Assign Test split(s) for use in Dataloaders
        if stage == "test":
            self.mnist_test = MNIST(self.data_dir, train=False, download=True, transform=self.transform)

`setup()`方法需要一个`stage`参数，它用来区分`trainer.{fit,validate,test,predict}`状态的`setup`逻辑。`setup`在每个节点的进程上都会调用，所以这里可以设置对象状态。

### train_dataloader
使用`train_dataloader()`方法生成训练数据加载器，这个加载器被`Trainer.fit()`方法使用。通常只需要使用在`setup`中定义的数据集。

In [ ]:
import pytorch_lightning as pl


class MNISTDataModule(pl.LightningDataModule):
    def train_dataloader(self):
        return DataLoader(self.mnist_train, batch_size=64)

### val_dataloader
操作和`train_dataloader()`相同，它生成的加载器在`Trainer.fit()`和`Trainer.validate()`中使用。
### test_dataloader
操作同上，它生成的加载器在`Trainer.test()`方法中使用。
### predict_dataloader
操作同上，它生成的加载器在`Trainer.predict()`方法中使用。

## 使用DataModule
使用DataModule的推荐方法非常简单：

In [ ]:
dm = MNISTDataModule()
model = Model()
trainer.fit(model, datamodule=dm)
trainer.test(datamodule=dm)
trainer.validate(datamodule=dm)
trainer.predict(datamodule=dm)

可以通过`trainer.datamodule`和`trainer.train_dataloader`等，获取当前使用的datamodule、训练数据加载器、验证集数据加载器等。

## DataModule中的超参数
和LightningModules相同，支持`save_hyperparameters()`方法。

In [ ]:
import pytorch_lightning as pl


class CustomDataModule(pl.LightningDataModule):
    def __init__(self, *args, **kwargs):
        super().__init__()
        self.save_hyperparameters()

    def configure_optimizers(self):
        # access the saved hyperparameters
        opt = optim.Adam(self.parameters(), lr=self.hparams.lr)